In [4]:
# !pip install opencv-python
# !pip install face_recognition
# !pip install numpy
# !pip install mediapipe opencv-python face_recognition
# !pip install Pillow numpy opencv-python albumentations
# !pip install opencv-python-headless face-recognition dlib matplotlib
# !pip install tqdm
# !pip install requests pillow
# !pip install pylance
# !pip install torch torchvision
# !pip install pandas requests beautifulsoup4
# !pip install mysql-connector-python
# !pip install requests
# # (Optional: install dlib separately if face_recognition does not handle it)


In [1]:
import mysql.connector
from mysql.connector import Error

def create_connection():
    try:
        connection = mysql.connector.connect(
            host='localhost',          # Replace with your host
            user='root',      # Replace with your MySQL username
            password='W7301@jqir#',  # Replace with your MySQL password
            database='FRAT_Attendance'
        )
        if connection.is_connected():
            print("Connected to MySQL database")
        return connection
    except Error as e:
        print(f"Error: '{e}'")
        return None


In [2]:

import os
import cv2
import numpy as np
import mediapipe as mp
import face_recognition
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from datetime import datetime
import requests

# Initialize Mediapipe face detection
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.7)

def load_known_faces(known_faces_dir):
    known_face_encodings = []
    known_face_names = []
    for filename in os.listdir(known_faces_dir):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(known_faces_dir, filename)
            image = face_recognition.load_image_file(image_path)
            encodings = face_recognition.face_encodings(image)
            if encodings:
                known_face_encodings.append(encodings[0])
                known_face_names.append(os.path.splitext(filename)[0])
            else:
                print(f"No face found in {filename}. Skipping.")
    return known_face_encodings, known_face_names

def register_student(known_faces_dir):
    connection = create_connection()
    cursor = connection.cursor()
    
    student_name = input("Enter the name of the new student: ")
    roll_no = input("Enter roll number: ")
    age = int(input("Enter age: "))
    phone_number = input("Enter phone number: ")
    email = input("Enter email: ")
    branch = input("Enter branch: ")
    college = input("Enter college: ")
    year_of_study = int(input("Enter year of study: "))
    course = input("Enter course/study stream: ")

    video_capture = cv2.VideoCapture(0)
    images_captured = 0
    img_path = os.path.join(known_faces_dir, f"{student_name}.jpg")

    while images_captured < 10:
        ret, frame = video_capture.read()
        if not ret:
            print("Failed to grab frame")
            break
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(rgb_frame)

        if results.detections:
            cv2.imwrite(img_path, frame)
            images_captured += 1
            print(f"Captured {images_captured} images for {student_name}")

    video_capture.release()

    cursor.execute(""" 
        INSERT INTO students (name, roll_no, age, phone_number, email, branch, college, year_of_study, course, image_path) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (student_name, roll_no, age, phone_number, email, branch, college, year_of_study, course, img_path))
    connection.commit()
    cursor.close()
    connection.close()
    print(f"Finished registering {student_name}. Details stored in the database.")

def take_attendance(known_face_encodings, known_face_names):
    connection = create_connection()
    cursor = connection.cursor()
    
    video_capture = cv2.VideoCapture(0)
    attendance = set()

    while True:
        ret, frame = video_capture.read()
        if not ret:
            print("Failed to grab frame")
            break
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(rgb_frame)
        face_locations = []

        if results.detections:
            for detection in results.detections:
                bboxC = detection.location_data.relative_bounding_box
                h, w, _ = rgb_frame.shape
                x = int(bboxC.xmin * w)
                y = int(bboxC.ymin * h)
                width = int(bboxC.width * w)
                height = int(bboxC.height * h)
                face_locations.append((y, x + width, y + height, x))

        if face_locations:
            face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

            for face_encoding, (top, right, bottom, left) in zip(face_encodings, face_locations):
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Unknown"
                roll_no = None

                if True in matches:
                    best_match_index = np.argmin(face_recognition.face_distance(known_face_encodings, face_encoding))
                    name = known_face_names[best_match_index]

                    # Fetch roll_no from the database
                    cursor.execute("SELECT roll_no FROM students WHERE name = %s", (name,))
                    result = cursor.fetchone()
                    if result:
                        roll_no = result[0]
                        attendance.add((name, roll_no))

                        # Insert attendance record into the database
                        cursor.execute("SELECT id FROM students WHERE name = %s", (name,))
                        student_id = cursor.fetchone()[0]
                        
                        current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        cursor.execute(""" 
                            INSERT INTO attendance (student_id, date, status) 
                            VALUES (%s, %s, %s)
                        """, (student_id, current_timestamp, 'Present'))
                        
                        # Fetch the email address of the student
                        cursor.execute("SELECT email FROM students WHERE id = %s", (student_id,))
                        result = cursor.fetchone()
                        if result:
                            student_email = result[0]
                            send_email_notification(name, student_email)

                        connection.commit()
                    else:
                        print(f"No roll number found for {name}")

                # Display name and roll_no on the frame
                display_text = f"{name} ({roll_no})" if roll_no else name
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                cv2.putText(frame, display_text, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        display(plt.gcf())
        clear_output(wait=True)

        action = input("Type 'end' to stop attendance or press Enter to continue: ").lower()
        if action == 'end':
            break

    plt.close()
    video_capture.release()
    cursor.close()
    connection.close()
    print("Attendance taken and logged in the database for:", attendance)



def send_email_notification(student_name, student_email):
    api_url = 'http://localhost:5000/api/send-email'
    payload = {
        'studentName': student_name,
        'studentEmail': student_email
    }
    response = requests.post(api_url, json=payload)
    if response.status_code == 200:
        print(f"Email sent to {student_name}")
    else:
        print("Failed to send email")

# # Example usage after attendance is captured
# send_email_notification("John Doe", "john.doe@example.com")


if __name__ == "__main__":
    known_faces_dir = 'Class_Attendance/Students'

    while True:
        action = input("Type 'register' to register a new student or 'attendance' to take attendance (or 'exit' to quit): ").lower()
        
        if action == 'register':
            register_student(known_faces_dir)
        elif action == 'attendance':
            known_face_encodings, known_face_names = load_known_faces(known_faces_dir)
            take_attendance(known_face_encodings, known_face_names)
        elif action == 'exit':
            print("Exiting program.")
            break
        else:
            print("Invalid action. Please try again.")

Attendance taken and logged in the database for: {('Gautam', '322506402458')}
Exiting program.


In [5]:
# 1. Load Known Faces :
# Time Complexity: O(n) - where n is the number of images in the directory.
# Space Complexity: O(n) - for storing face encodings and names.

# 2. Register New Student :
# Time Complexity: O(1) - fixed number of images 10 captured, but processing each frame can vary.
# Space Complexity: O(1) - for temporary variables and images captured.

# 3. Take Attendance :
# Time Complexity: O(d . n) - where d is the number of detected faces, and n is the number of known faces.
# Space Complexity: O(d) - for storing detected face locations and encodings.

# Overall : 
# Overall Time Complexity: Dominated by attendance function, O(d . n).
# Overall Space Complexity: O(n) - primarily for known face encodings.
